In [8]:
import requests
from typing import Optional, List, Dict, Any
from dotenv import load_dotenv
import os

import together
import openai
import json

load_dotenv()
google_api_key = os.getenv('google_api_key')

In [5]:
def get_all_nearby_places(lat: float = 37.7937, long: float = -122.3965, fields: Optional[List[str]] = None) -> requests.Response:
    if fields is None:
        fields = ["displayName", "editorialSummary"]
    
    fields = ["places." + field for field in fields]

    url = "https://places.googleapis.com/v1/places:searchNearby"
    query = {
            "includedTypes": ["restaurant"],
            "maxResultCount": 10,
            "locationRestriction": {
                "circle": {
                "center": {
                    "latitude": lat,
                    "longitude": long},
                "radius": 500.0
                }
            }
        }
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": google_api_key,
        "X-Goog-FieldMask": ",".join(fields)
    }
    response = requests.post(url, data=json.dumps(query), headers=headers)

    return response

In [6]:
response = get_nearby_places()

In [10]:
json_res = response.json()

print(json_res)

{'places': [{'displayName': {'text': 'La Mar Cocina Peruana', 'languageCode': 'en'}, 'editorialSummary': {'text': 'Celeb chef Gastón Acurio serves Peruvian seafood in a large, partially outdoor space on the pier.', 'languageCode': 'en'}}, {'displayName': {'text': 'Kokkari Estiatorio', 'languageCode': 'en'}, 'editorialSummary': {'text': 'Taverna-inspired cuisine served in a candlelit, wood-beamed dining room that attracts a chic crowd.', 'languageCode': 'en'}}, {'displayName': {'text': 'Harborview Restaurant & Bar', 'languageCode': 'en'}, 'editorialSummary': {'text': 'Contemporary, family-style outlet serving Cantonese-style dishes with panoramic harbor views.', 'languageCode': 'en'}}, {'displayName': {'text': 'Hog Island Oyster Co.', 'languageCode': 'en'}, 'editorialSummary': {'text': 'Contemporary waterside seafood eatery serving local oysters & clams, plus craft cocktails.', 'languageCode': 'en'}}, {'displayName': {'text': 'Barcha Restaurant', 'languageCode': 'en'}, 'editorialSummary

In [ ]:
def is_relevant_field(field: str = "servesVegetarianFood", pref: str = "Looking for vegetarian food, Asian cuisine") -> bool:
    pass

def is_relevant_place(place_dict: Dict[str, Any], pref: str = "Looking for vegetarian food, Asian cuisine") -> bool:
    pass

def get_relevant_nearby_places(lat: float = 37.7937, long: float = -122.3965, pref: str = "Looking for a vegan restaurant that is student budget friendly.") -> Optional[List[Dict[str, Any]]]:
    # construct fields for the query
    required_fields = ["displayName", "editorialSummary"]
    base_fields = ["accessibilityOptions", "addressComponents", "adrFormatAddress", "businessStatus", "formattedAddress", "googleMapsUri", "iconBackgroundColor", "iconMaskBaseUri", "id", "location", "name*", "photos", "plusCode", "primaryType", "primaryTypeDisplayName", "shortFormattedAddress", "subDestinations", "types", "utcOffsetMinutes", "viewport"]
    adv_fields = ["currentOpeningHours", "currentSecondaryOpeningHours", "internationalPhoneNumber", "nationalPhoneNumber", "priceLevel", "rating", "regularOpeningHours", "regularSecondaryOpeningHours", "userRatingCount", "websiteUri"]
    pref_fields = ["allowsDogs", "curbsidePickup", "delivery", "dineIn", "evChargeOptions*", "fuelOptions*", "goodForChildren", "goodForGroups", "goodForWatchingSports", "liveMusic", "menuForChildren", "parkingOptions*", "paymentOptions", "outdoorSeating", "reservable", "restroom", "reviews", "servesBeer", "servesBreakfast", "servesBrunch", "servesCocktails", "servesCoffee", "servesDesserts", "servesDinner", "servesLunch", "servesVegetarianFood", "servesWine", "takeout"]
    all_optional_fields = base_fields + adv_fields + pref_fields
    
    filtered_fields = required_fields + [field for field in all_optional_fields if is_relevant_field(field, pref)]

    # perform query to find all nearby places with the above fields
    all_nearby_places = get_all_nearby_places(lat=lat, long=long, fields=filtered_fields)
    if all_nearby_places.status_code == 200:
        # Parse the response to JSON
        places_dict = all_nearby_places.json()
    else:
        print("Error:", all_nearby_places.status_code, all_nearby_places.reason)
        return
    # use the above reponse descriptions and user pref. to filter relevant places
    filtered_places = []
    for place_dict in places_dict["places"]:
        if is_relevant_place(place_dict, pref):
            filtered_places.append(place_dict)
    # return the relevant places along with their fields
    return filtered_places
